In [1]:
# import dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd 
import json
import time
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# import course df and isolate course names
nc_courses_df = pd.read_csv("../data/nc_courses_df.csv")

# empty variable to append addresses to
course_address = []

# loop to call all courses
for index, row in nc_courses_df.iterrows():
    url = "https://www.pdga.com/course-directory/advanced"
    browser.visit(url)
    time.sleep(4)
    browser.fill("title", row[4])
    time.sleep(3)
    browser.find_option_by_text("United States").first.click()
    time.sleep(3)
    browser.fill("field_course_location_locality", row[2])
    time.sleep(3)
    browser.find_option_by_text("North Carolina").first.click()
    browser.find_by_id('edit-submit-course-directory').first.click()
    time.sleep(6)
    try:
        course_href = browser.links.find_by_text(row[4])['href']
        browser.links.find_by_text(row[4]).click()
    except ElementDoesNotExist:
        try:
            course_href = browser.links.find_by_partial_text(row[4])['href']
            browser.links.find_by_partial_text(row[4]).click()
        except:  
            course_address.append("Not Available")
            print(row[4] + " address not available")
                
    time.sleep(4)
    html = requests.get(course_href)
    soup = bs(html.text, 'html.parser')
    try:
        street = soup.find('div', class_="thoroughfare").text
        city = soup.find('span', class_="locality").text
        state = soup.find('span', class_="state").text
        address = f'{street}, {city}, {state}'
        course_address.append(address)
        print(row[4] + " address has been gathered")
    except:  
        course_address.append("Not Available")
        print(row[4] + " address not available")


Ahoskie Creek Park address has been gathered
Alamance Reserve Apartments address has been gathered
Anderson Creek address has been gathered
Anderson Park Disc Golf address has been gathered
Apex Nature Park address has been gathered
Armfield Civic Center address has been gathered
Arnette Park address has been gathered
Arrowhead Park address has been gathered
Ashe County Park Disc Golf Course address has been gathered
Avalon Peaks Apartments address has been gathered
Ayden Park address has been gathered
Bailey Road Park address has been gathered
Barber Park address has been gathered
Barnet Park address has been gathered
Bear Creek Disc Golf Course address has been gathered
Bear Lake Reserve Disc Golf Course address has been gathered
Bearskin Creek at Dickerson Park address has been gathered
Beatty's Ford Park address has been gathered
Beaufort County Community College Disc Golf Course address has been gathered
Beech Mountain Resort Disc Golf address not available
Beech Mountain Resort D

Palmer Park Disc Golf Course address has been gathered
Persimmon Hollar address has been gathered
Piedmont Natural Gas address has been gathered
Pine Cone Crush at Reservoir Park address has been gathered
Pinewood Elementary Disc Golf Course address has been gathered
Piney Lake Disc Golf Course address has been gathered
Pisgah Brewery address has been gathered
Plantation Ruins at Winget address has been gathered
Pleasant Grove Community Center address has been gathered
Pleasant Grove Disc Golf Course address has been gathered
Potter's Grove at Presnell Park address has been gathered
Rankin Lake Disc Golf Course address has been gathered
Re3 Disc Golf Course address has been gathered
Redwood Park address has been gathered
Reedy Creek Park address has been gathered
Renaissance Park - Gold address has been gathered
Renaissance Park - Pro Players address has been gathered
Renaissance Park - RenSke address has been gathered
Rescue Squad Park address has been gathered
Resurrection Park addre

In [14]:
# append list to nc_courses_df
nc_courses_df["Course Address"] = course_address

In [15]:
# save updated dataframe
nc_courses_df.to_csv("../data/nc_courses_df.csv")